<a href="https://colab.research.google.com/github/gcicconofri098/Ice_Cube_project_CMEPDA/blob/main/project_notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

#plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
#plt.rc('figure', autolayout=True)
#plt.rc('axes', labelweight='bold', labelsize='large',
#       titleweight='bold', titlesize=18, titlepad=10)
#plt.rc('animation', html='html5')


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

def angular_dist_score(y_true, y_pred):
    az_true = y_true[:,0]
    zen_true = y_true[:,1]
    az_pred = y_pred[:,0]
    zen_pred = y_pred[:,1]
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)

    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)

    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)

    # scalar product of two unit vectors is always between -1 and 1, this is against numerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod =  np.clip(scalar_prod, -1, 1)

    # convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))

![image.png](attachment:2c7dbbb6-0aa1-4f57-aae0-5f209f5c2600.png)

In [ ]:
#Read files
df = pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/train/batch_2.parquet').reset_index()

res = pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/train_meta.parquet')

geom = pd.read_csv('/kaggle/input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv')

#Filters on events

df = df[df['auxiliary']==False]

df = df.drop(labels='auxiliary', axis=1)

df.head()


,event_id,sensor_id,time,charge
12,3266199,1775,9877,0.575
13,3266199,1775,9882,1.225
14,3266199,1775,9888,0.975
15,3266199,1775,9893,0.975
16,3266199,1775,9908,1.025


In [ ]:
#Geometrical stuff

geom['sensor_azimuth'] = np.arctan(geom['y']/geom['x'])
geom['sensor_zenith'] = np.arctan(np.sqrt(geom['x']**2 + geom['y']**2)/geom['z'])

geom.head()


,sensor_id,x,y,z,sensor_azimuth,sensor_zenith
0,0,-256.14,-521.08,496.03,1.113927,0.863814
1,1,-256.14,-521.08,479.01,1.113927,0.881007
2,2,-256.14,-521.08,461.99,1.113927,0.898702
3,3,-256.14,-521.08,444.97,1.113927,0.916909
4,4,-256.14,-521.08,427.95,1.113927,0.935639


In [ ]:
#add spacial positions of sensor on dataframe

df['charge'].astype(np.float16()) 
df_with_geom = df.merge(geom, how = 'left', on = 'sensor_id').reset_index(drop = True)
#df_with_geom = df_with_geom.merge(res, how = 'left', on = 'event_id').reset_index(drop = True)
#res = res.drop(labels = ['batch_id', 'first_pulse_index', 'last_pulse_index'], axis= 1)
#df_with_geom = df_with_geom.drop(['x', 'y', 'z'], axis = 1)
df_with_geom.head(15)


,event_id,sensor_id,time,charge,x,y,z,sensor_azimuth,sensor_zenith
0,3266199,1775,9877,0.575,500.43,-58.45,-93.87,-0.116273,-1.386596
1,3266199,1775,9882,1.225,500.43,-58.45,-93.87,-0.116273,-1.386596
2,3266199,1775,9888,0.975,500.43,-58.45,-93.87,-0.116273,-1.386596
3,3266199,1775,9893,0.975,500.43,-58.45,-93.87,-0.116273,-1.386596
4,3266199,1775,9908,1.025,500.43,-58.45,-93.87,-0.116273,-1.386596
5,3266199,1776,9912,0.725,500.43,-58.45,-110.89,-0.116273,-1.354157
6,3266199,1775,9916,0.975,500.43,-58.45,-93.87,-0.116273,-1.386596
7,3266199,1775,9925,0.975,500.43,-58.45,-93.87,-0.116273,-1.386596
8,3266199,1776,9966,1.575,500.43,-58.45,-110.89,-0.116273,-1.354157
9,3266199,1775,9988,0.875,500.43,-58.45,-93.87,-0.116273,-1.386596


In [ ]:
del df

In [ ]:
df_with_geom = df_with_geom.drop(labels=['x', 'y', 'z'], axis=1)

In [ ]:
#df_with_geom = df_with_geom.groupby(['event_id']).apply(lambda s: s.sort_values(by='time')).reset_index(drop = True)
df_with_geom = df_with_geom.sort_values('charge', ascending = False).drop_duplicates(['event_id', 'sensor_id']).sort_index()


df_with_geom.head(25)

,event_id,sensor_id,time,charge,sensor_azimuth,sensor_zenith
11,3266199,1775,10004,1.375,-0.116273,-1.386596
13,3266199,1776,10047,1.925,-0.116273,-1.354157
14,3266199,1774,10059,1.075,-0.116273,-1.419432
17,3266199,1166,10969,1.475,-0.569066,1.427733
19,3266199,1105,11028,1.875,-0.783732,1.334892
21,3266199,1106,11034,1.025,-0.783732,1.389699
22,3266199,1107,11066,1.275,-0.783732,1.445661
26,3266199,1167,11122,1.625,-0.569066,1.470974
27,3266199,1104,11173,1.275,-0.783732,1.281491
30,3266199,1109,11932,0.925,-0.783732,1.559557


In [ ]:
df_with_geom['n_counter'] = df_with_geom.groupby('event_id').cumcount()
df_with_geom

,event_id,sensor_id,time,charge,sensor_azimuth,sensor_zenith,n_counter
11,3266199,1775,10004,1.375,-0.116273,-1.386596,0
13,3266199,1776,10047,1.925,-0.116273,-1.354157,1
14,3266199,1774,10059,1.075,-0.116273,-1.419432,2
17,3266199,1166,10969,1.475,-0.569066,1.427733,3
19,3266199,1105,11028,1.875,-0.783732,1.334892,4
...,...,...,...,...,...,...,...
22726668,6517807,2333,12357,0.675,0.031557,-0.799732,12
22726669,6517807,1735,12358,1.175,-0.243179,-0.723330,13
22726670,6517807,2337,12365,1.475,0.031557,-0.721389,14
22726671,6517807,1737,12462,1.025,-0.243179,-0.686039,15


In [ ]:
df_with_geom  = df_with_geom[df_with_geom.n_counter<100]
df_with_geom

,event_id,sensor_id,time,charge,sensor_azimuth,sensor_zenith,n_counter
11,3266199,1775,10004,1.375,-0.116273,-1.386596,0
13,3266199,1776,10047,1.925,-0.116273,-1.354157,1
14,3266199,1774,10059,1.075,-0.116273,-1.419432,2
17,3266199,1166,10969,1.475,-0.569066,1.427733,3
19,3266199,1105,11028,1.875,-0.783732,1.334892,4
...,...,...,...,...,...,...,...
22726668,6517807,2333,12357,0.675,0.031557,-0.799732,12
22726669,6517807,1735,12358,1.175,-0.243179,-0.723330,13
22726670,6517807,2337,12365,1.475,0.031557,-0.721389,14
22726671,6517807,1737,12462,1.025,-0.243179,-0.686039,15


In [ ]:
maxima = df_with_geom.groupby('event_id')['n_counter'].max().values
maxima

array([ 9, 11,  5, ..., 13,  5, 16])

In [ ]:
n_counter_1 = np.where(maxima > 99, maxima, 99 )
n_counter_1

array([99, 99, 99, ..., 99, 99, 99])

In [ ]:
diff = n_counter_1 - maxima
diff

array([90, 88, 94, ..., 86, 94, 83])

In [ ]:
df_with_geom.set_index(['event_id', 'n_counter'], inplace= True)
df_with_geom

sensor_id   time  charge  sensor_azimuth  sensor_zenith
event_id n_counter                                                         
3266199  0               1775  10004   1.375       -0.116273      -1.386596
         1               1776  10047   1.925       -0.116273      -1.354157
         2               1774  10059   1.075       -0.116273      -1.419432
         3               1166  10969   1.475       -0.569066       1.427733
         4               1105  11028   1.875       -0.783732       1.334892
...                       ...    ...     ...             ...            ...
6517807  12              2333  12357   0.675        0.031557      -0.799732
         13              1735  12358   1.175       -0.243179      -0.723330
         14              2337  12365   1.475        0.031557      -0.721389
         15              1737  12462   1.025       -0.243179      -0.686039
         16              1792  12567   0.975       -0.116273      -0.920539

[3531144 rows x 5 columns]

In [ ]:
n_rows = np.sum(diff)
ev_ids = np.unique(df_with_geom.index.get_level_values(0).values)
zeros = np.zeros((n_rows, 5), dtype = np.int32)
new_index = np.repeat(ev_ids, diff)
new_index

array([3266199, 3266199, 3266199, ..., 6517807, 6517807, 6517807])

In [ ]:
pad_df = pd.DataFrame(zeros, index = new_index ,columns=df_with_geom.columns).reset_index(drop = False)
pad_df['event_id'] = pad_df['index']
pad_df = pad_df.drop(labels = ['index'], axis =1)
pad_df['n_counter'] = pad_df.groupby('event_id').cumcount() + df_with_geom.groupby('event_id').cumcount().max() +1
 
#pad_df = pad_df.set_index(['event_id', 'n_counter'])
#pad_df = pad_df.drop(labels = 'index', axis = 1)

pad_df.head()

,sensor_id,time,charge,sensor_azimuth,sensor_zenith,event_id,n_counter
0,0,0,0,0,0,3266199,100
1,0,0,0,0,0,3266199,101
2,0,0,0,0,0,3266199,102
3,0,0,0,0,0,3266199,103
4,0,0,0,0,0,3266199,104


In [ ]:
pad_df = pad_df.set_index(['event_id', 'n_counter'])
pad_df

sensor_id  time  charge  sensor_azimuth  sensor_zenith
event_id n_counter                                                        
3266199  100                0     0       0               0              0
         101                0     0       0               0              0
         102                0     0       0               0              0
         103                0     0       0               0              0
         104                0     0       0               0              0
...                       ...   ...     ...             ...            ...
6517807  178                0     0       0               0              0
         179                0     0       0               0              0
         180                0     0       0               0              0
         181                0     0       0               0              0
         182                0     0       0               0              0

[16468856 rows x 5 columns]

In [ ]:
df_final = pd.concat([df_with_geom, pad_df])
df_final.head(20)

sensor_id   time  charge  sensor_azimuth  sensor_zenith
event_id n_counter                                                         
3266199  0               1775  10004   1.375       -0.116273      -1.386596
         1               1776  10047   1.925       -0.116273      -1.354157
         2               1774  10059   1.075       -0.116273      -1.419432
         3               1166  10969   1.475       -0.569066       1.427733
         4               1105  11028   1.875       -0.783732       1.334892
         5               1106  11034   1.025       -0.783732       1.389699
         6               1107  11066   1.275       -0.783732       1.445661
         7               1167  11122   1.625       -0.569066       1.470974
         8               1104  11173   1.275       -0.783732       1.281491
         9               1109  11932   0.925       -0.783732       1.559557
3266228  0               3897   9899   1.175        1.065710      -0.667515
         1               3896   9947   1.225        1.065710      -0.686027
         2               3898  10020   1.225        1.065710      -0.649834
         3               3894  10037   2.675        1.065710      -0.725729
         4               3410  10656   1.075        0.688363      -0.765643
         5               3899  10766   1.075        1.065710      -0.632939
         6               3941  10970   1.375        0.835790      -1.182844
         7               3940  11051   1.025        0.835790      -1.215497
         8               3409  11404   1.225        0.688363      -0.790814
         9               3980  12301   0.275        0.684890       1.280561

In [ ]:
# sort by index
df_final = df_final.sort_index(level = 0)
#df_final = df_final.reset_index(level= 'n_counter')
df_final.head(20)

sensor_id   time  charge  sensor_azimuth  sensor_zenith
event_id n_counter                                                         
3266199  0               1775  10004   1.375       -0.116273      -1.386596
         1               1776  10047   1.925       -0.116273      -1.354157
         2               1774  10059   1.075       -0.116273      -1.419432
         3               1166  10969   1.475       -0.569066       1.427733
         4               1105  11028   1.875       -0.783732       1.334892
         5               1106  11034   1.025       -0.783732       1.389699
         6               1107  11066   1.275       -0.783732       1.445661
         7               1167  11122   1.625       -0.569066       1.470974
         8               1104  11173   1.275       -0.783732       1.281491
         9               1109  11932   0.925       -0.783732       1.559557
         100                0      0   0.000        0.000000       0.000000
         101                0      0   0.000        0.000000       0.000000
         102                0      0   0.000        0.000000       0.000000
         103                0      0   0.000        0.000000       0.000000
         104                0      0   0.000        0.000000       0.000000
         105                0      0   0.000        0.000000       0.000000
         106                0      0   0.000        0.000000       0.000000
         107                0      0   0.000        0.000000       0.000000
         108                0      0   0.000        0.000000       0.000000
         109                0      0   0.000        0.000000       0.000000

In [ ]:
df_final['counter'] = df_final.groupby('event_id').cumcount() 
df_final.head(30)

sensor_id   time  charge  sensor_azimuth  sensor_zenith  \
event_id n_counter                                                            
3266199  0               1775  10004   1.375       -0.116273      -1.386596   
         1               1776  10047   1.925       -0.116273      -1.354157   
         2               1774  10059   1.075       -0.116273      -1.419432   
         3               1166  10969   1.475       -0.569066       1.427733   
         4               1105  11028   1.875       -0.783732       1.334892   
         5               1106  11034   1.025       -0.783732       1.389699   
         6               1107  11066   1.275       -0.783732       1.445661   
         7               1167  11122   1.625       -0.569066       1.470974   
         8               1104  11173   1.275       -0.783732       1.281491   
         9               1109  11932   0.925       -0.783732       1.559557   
         100                0      0   0.000        0.000000       0.000000   
         101                0      0   0.000        0.000000       0.000000   
         102                0      0   0.000        0.000000       0.000000   
         103                0      0   0.000        0.000000       0.000000   
         104                0      0   0.000        0.000000       0.000000   
         105                0      0   0.000        0.000000       0.000000   
         106                0      0   0.000        0.000000       0.000000   
         107                0      0   0.000        0.000000       0.000000   
         108                0      0   0.000        0.000000       0.000000   
         109                0      0   0.000        0.000000       0.000000   
         110                0      0   0.000        0.000000       0.000000   
         111                0      0   0.000        0.000000       0.000000   
         112                0      0   0.000        0.000000       0.000000   
         113                0      0   0.000        0.000000       0.000000   
         114                0      0   0.000        0.000000       0.000000   
         115                0      0   0.000        0.000000       0.000000   
         116                0      0   0.000        0.000000       0.000000   
         117                0      0   0.000        0.000000       0.000000   
         118                0      0   0.000        0.000000       0.000000   
         119                0      0   0.000        0.000000       0.000000   

                    counter  
event_id n_counter           
3266199  0                0  
         1                1  
         2                2  
         3                3  
         4                4  
         5                5  
         6                6  
         7                7  
         8                8  
         9                9  
         100             10  
         101             11  
         102             12  
         103             13  
         104             14  
         105             15  
         106             16  
         107             17  
         108             18  
         109             19  
         110             20  
         111             21  
         112             22  
         113             23  
         114             24  
         115             25  
         116             26  
         117             27  
         118             28  
         119             29

In [ ]:
df_final = df_final.set_index(['event_id', 'counter'])
df_final.head(20)

KeyError: "None of ['event_id'] are in the columns"

In [ ]:
del df_with_geom

In [ ]:
df_final.head(10)

time  charge  sensor_azimuth  sensor_zenith
event_id                                                
3266199  0  10004   1.375       -0.116273      -1.386596
         1      0   0.000        0.000000       0.000000
         2      0   0.000        0.000000       0.000000
         3      0   0.000        0.000000       0.000000
         4      0   0.000        0.000000       0.000000
         5      0   0.000        0.000000       0.000000
         6      0   0.000        0.000000       0.000000
         7      0   0.000        0.000000       0.000000
         8      0   0.000        0.000000       0.000000
         9      0   0.000        0.000000       0.000000

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
3266199,17,53159869,10814344,10814394,3.548180,1.127325
3266228,17,53160228,10816254,10816298,5.036001,2.185961
3266282,17,53161017,10820603,10820849,3.483392,1.291909
3266329,17,53161812,10824429,10824471,2.483661,0.659159
3266370,17,53162515,10830603,10830664,0.281627,2.708783
...,...,...,...,...,...,...
6517762,33,106037408,18621691,18621747,1.932869,0.828631
6517770,33,106037534,18622393,18622433,4.424092,2.574416
6517775,33,106037665,18622792,18622862,3.450834,2.193151
6517794,33,106038020,18624232,18624340,5.193504,2.408181


In [ ]:
df_final1 = df_final.unstack(level = -1)
df_final1.head(20)
#df_final1.columns

sensor_id                                                        ...  \
                0     1     2     3     4     5     6     7     8     9   ...   
event_id                                                                  ...   
3266199       1775  1776  1774  1166  1105  1106  1107  1167  1104  1109  ...   
3266228       3897  3896  3898  3894  3410  3899  3941  3940  3409  3980  ...   
3266282        732  1143   734  1141  1142  1140     0     0     0     0  ...   
3266329       2380  2381  2383  2384  2382  2387  2995  2388  2990  2996  ...   
3266370       4363  4362  4364  4360  3940  3941     0     0     0     0  ...   
3266371       2713  2715  2712  2173  2174     0     0     0     0     0  ...   
3266394       4022  4024  4023  4021  4025  4029  4028     0     0     0  ...   
3266395       2765  2762  2174  2176  2763  1645  1643  1641     0     0  ...   
3266426       4321  4324  4320  4325  4269  4272  4326  4273  4268  4270  ...   
3266435        592  1069  1068   593  1666  1665  4957  1731  1730  1664  ...   
3266439       3301  3300  3303  3302  3305  2763  3367  2830  2832  2833  ...   
3266442       2819  2818  2817  2816  2813  2811     0     0     0     0  ...   
3266448       3523  3521  2235  2234  2236  2233  2237  2238  2230  2232  ...   
3266478       4343  4342  4340  4341     0     0     0     0     0     0  ...   
3266484       3943  3946  4007  4002  3464  3463  3940  3950  3948  3462  ...   
3266488       3788  3791  3790  3795  3797  3748  3750  3752  3747  3745  ...   
3266508       3147  2549  2608  3150  2548  2606  2547  3148  1403  1402  ...   
3266525       3071  3070  3233  3296  3298  3232  2757  2755  2953  2357  ...   
3266526       4435  4437  4378  4377     0     0     0     0     0     0  ...   
3266529        261   262   639   642   645   644   281   646   648   283  ...   

         sensor_zenith                                               
                    90   91   92   93   94   95   96   97   98   99  
event_id                                                             
3266199            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266228            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266282            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266329            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266370            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266371            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266394            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266395            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266426            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266435            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266439            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266442            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266448            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266478            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266484            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266488            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266508            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266525            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266526            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3266529            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[20 rows x 500 columns]

In [ ]:
df_final.head(20)

sensor_id   time  charge  sensor_azimuth  sensor_zenith
event_id                                                            
3266199  0        1775  10004   1.375       -0.116273      -1.386596
         1        1776  10047   1.925       -0.116273      -1.354157
         2        1774  10059   1.075       -0.116273      -1.419432
         3        1166  10969   1.475       -0.569066       1.427733
         4        1105  11028   1.875       -0.783732       1.334892
         5        1106  11034   1.025       -0.783732       1.389699
         6        1107  11066   1.275       -0.783732       1.445661
         7        1167  11122   1.625       -0.569066       1.470974
         8        1104  11173   1.275       -0.783732       1.281491
         9        1109  11932   0.925       -0.783732       1.559557
         10          0      0   0.000        0.000000       0.000000
         11          0      0   0.000        0.000000       0.000000
         12          0      0   0.000        0.000000       0.000000
         13          0      0   0.000        0.000000       0.000000
         14          0      0   0.000        0.000000       0.000000
         15          0      0   0.000        0.000000       0.000000
         16          0      0   0.000        0.000000       0.000000
         17          0      0   0.000        0.000000       0.000000
         18          0      0   0.000        0.000000       0.000000
         19          0      0   0.000        0.000000       0.000000

In [ ]:
res  = res[res.event_id == df_final.index.get_level_values(0)]

res = res.sort_index()
res

ValueError: ('Lengths must match to compare', (131953924,), (20000000,))

In [ ]:
df_final1 = df_final1.reset_index(drop = False)
df_final1

event_id sensor_id                                                  \
                        0     1     2     3     4     5     6     7     8   
0       3266199      1775  1776  1774  1166  1105  1106  1107  1167  1104   
1       3266228      3897  3896  3898  3894  3410  3899  3941  3940  3409   
2       3266282       732  1143   734  1141  1142  1140     0     0     0   
3       3266329      2380  2381  2383  2384  2382  2387  2995  2388  2990   
4       3266370      4363  4362  4364  4360  3940  3941     0     0     0   
...         ...       ...   ...   ...   ...   ...   ...   ...   ...   ...   
199995  6517762      4637  4639  4635  4640  4636  4633  4580  4634  4584   
199996  6517770        65    64    63    62    66    61   553   552   555   
199997  6517775      2529  2530  2528  2526  2525  2532  3547  3549  3548   
199998  6517794      3790  3791  3787  3788  3792  3793     0     0     0   
199999  6517807      2961  2963  2964  2965  2370  2368  2328  2331  2334   

        ... sensor_zenith                                               
        ...            90   91   92   93   94   95   96   97   98   99  
0       ...           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1       ...           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2       ...           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3       ...           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4       ...           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...     ...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
199995  ...           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
199996  ...           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
199997  ...           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
199998  ...           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
199999  ...           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[200000 rows x 501 columns]

In [ ]:
#Defining the targets and the variables

X = df_final1.loc[:,['time','charge','sensor_azimuth', 'sensor_zenith']] #variables to be taken from the batch
Y = res.loc[:,['azimuth', 'zenith']] #target for the training to be taken from the metadata file


In [ ]:
plt.hist((df_with_geom.groupby(by="event_id").count())["zenith"],bins=100,range=(0,100))
plt.yscale("log")

NameError: name 'df_with_geom' is not defined

In [ ]:
#Study the distribution of the variables and targets

plt.hist(df_with_geom['time'], bins = 100)
plt.xlabel("time")

In [ ]:
X.shape

(200000, 792)

In [ ]:
plt.hist(df_with_geom['charge'], bins=100)
plt.xlabel("charge")
plt.yscale('log1p')

In [ ]:
model = keras.Sequential([layers.Dense(units=512, input_shape= [400], activation = 'relu'),
                         layers.Dense(units = 512, activation = 'relu'),
                         layers.Dense(units = 512, activation = 'relu'),
                         layers.Dense(units = 512, activation = 'relu'),
                         layers.Dense(units = 512, activation = 'relu'),
                         layers.Dense(units= 2, activation = 'linear')])

predicted_X = model.predict(X)


model.compile(optimizer=Adam(learning_rate= 0.0001), loss = 'mae')#angular_dist_score(np.array(Y), np.array(predicted_X)))

In [ ]:
model.summary()

In [ ]:
history = model.fit(X, Y,batch_size=128,epochs=30, validation_split = 0.4)
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5. You can change this to get a different view.
history_df.loc[5:, ['loss', 'val_loss']].plot()


In [ ]:
print(X.shape)

In [ ]:
Y["azimuth"][0]